In [16]:
# 01. Import thư viện và load dữ liệu
import csv
import numpy as np

data_path = "../data/raw/BankChurners.csv"
# Đọc dữ liệu từ file CSV, tách header và dữ liệu
rows = []
with open(data_path, "r") as f:
    reader = csv.reader(f)
    header = next(reader)  # lưu tên cột
    for row in reader:
        rows.append(row)

data_raw = np.array(rows, dtype=object)

In [17]:
# 02. Xử lý giá trị thiếu (Unknown)

data = data_raw.copy()

# Thống kê giá trị "Unknown" cho tất cả các cột
print("- Thống kê giá trị chưa xác định (Unknown)")
for i, col in enumerate(header):
    col_vals = data[:, i]
    n_unknown = np.sum(col_vals == "Unknown")
    pct_unknown = n_unknown / len(col_vals) * 100
    if n_unknown > 0:
        print(f"{col}: {n_unknown} giá trị Unknown ({pct_unknown:.2f}%)")

# Điền giá trị missing bằng mode
for i, col in enumerate(header):
    col_vals = data[:, i]
    mask = col_vals == "Unknown"
    if np.any(mask):
        # Lấy mode của các giá trị hợp lệ (không phải Unknown)
        vals, counts = np.unique(col_vals[col_vals != "Unknown"], return_counts=True)
        mode_val = vals[np.argmax(counts)]
        data[mask, i] = mode_val


- Thống kê giá trị chưa xác định (Unknown)
Education_Level: 1519 giá trị Unknown (15.00%)
Marital_Status: 749 giá trị Unknown (7.40%)
Income_Category: 1112 giá trị Unknown (10.98%)


#### Bước này thay thế tất cả "Unknown" bằng giá trị xuất hiện nhiều nhất trong cột tương ứng.

In [18]:
# 03. Loại bỏ các cột không dùng

remove_cols = {
    "CLIENTNUM",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2",
}

keep_indices = [i for i, col in enumerate(header) if col not in remove_cols]
data = data[:, keep_indices]
header = [header[i] for i in keep_indices]

In [19]:
# 04. Tách target

target_idx = header.index("Attrition_Flag")

y = (data[:, target_idx] == "Attrited Customer").astype(int)  # 1 = Attrited, 0 = Existing

# loại bỏ cột target khỏi data
X_data = np.delete(data, target_idx, axis=1)
X_header = [h for h in header if h != "Attrition_Flag"]


In [20]:
# 05. One-hot encoding cho categorical

categorical_cols = ["Gender","Education_Level","Marital_Status","Income_Category","Card_Category"]

numeric_idx = []
categorical_idx = []

for h in X_header:
    if h in categorical_cols:
        categorical_idx.append(X_header.index(h))
    else:
        numeric_idx.append(X_header.index(h))

# Numeric
X_num = X_data[:, numeric_idx].astype(float)
num_header = [X_header[i] for i in numeric_idx]

# One-hot cho categorical
onehot_blocks = []
onehot_headers = []

for idx in categorical_idx:
    col = X_data[:, idx]
    uniq = np.unique(col)
    oh = np.zeros((len(col), len(uniq)))
    for j, u in enumerate(uniq):
        oh[:, j] = (col == u).astype(int)
    onehot_blocks.append(oh)
    onehot_headers += [f"{X_header[idx]}_{u}" for u in uniq]

# Kết hợp numeric + one-hot 
X = np.concatenate([X_num] + onehot_blocks, axis=1)
final_header = num_header + onehot_headers

In [21]:
# 06. Lưu file processed

out_csv = "../data/processed/BankChurners_processed.csv"

with open(out_csv, "w") as f:
    f.write(",".join(final_header) + ",target\n")
    for i in range(len(X)):
        f.write(",".join(map(str, X[i])) + "," + str(y[i]) + "\n")

print("Saved to:", out_csv)

Saved to: ../data/processed/BankChurners_processed.csv
